# Hudi example<br>




In [1]:
import org.apache.hadoop.fs.FileSystem
import org.apache.hudi.QuickstartUtils._
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.config.HoodieWriteConfig._
import scala.collection.JavaConversions._

In [2]:
// olist_orders dataset
val datasetPath = "s3://kiyoung-data-us-east-1/dataset/olist_orders_dataset"
val df = spark.read.load(datasetPath)

In [3]:
// 데이터는 잘 있습니다.
df.show

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|995392413cee61cc1...|4bf24904ec428325a...|   delivered|     2017-09-04 13:24:05|2017-09-04 13:43:54|         2017-09-13 08:20:04|          2017-09-22 12:09:32|          2017-09-26 15:00:00|
|b39de9ed2bb8fd08a...|ed18b557140ff674f...|   delivered|     2018-03-27 00:15:17|2018-03-27 00:30:12|         2018-03-27 14:52:09|          2018-04-11 12:51:02|          2018-04-25 15:00:00|
|b1a88554eb1f7f686...|5cf799d0ac88e1d32...|  

In [4]:
// 스키마는 이렇게 생겼습니다.
df.printSchema

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [5]:
// hudi 데이터를 저장할 path 와 table 이름 입니다.
val basePath = "s3://kiyoung-data-store-us-east-1/hudi/olist_orders_dataset_mor"
val dbName = "hudi"
val tableName = "olist_orders_mor"

// 이 녀석은 예제에 많이 나오는 녀석인데, 별 것은 아니고, hoodie의 parallelism 셋팅 입니다.
getQuickstartWriteConfigs

{hoodie.upsert.shuffle.parallelism=2, hoodie.insert.shuffle.parallelism=2}

In [7]:
// Insert 
// TABLE_TYPE_OPT_KEY 를 MERGE_ON_READ로 지정하였습니다.
df.withColumn("order_purchase_date", regexp_replace(date_format(to_date(col("order_purchase_timestamp")), "yyyy-MM-dd"), "-", "/"))
  .filter("order_purchase_date='2018/05/28' or order_purchase_date='2017/09/11' or order_purchase_date='2018/08/10' or order_purchase_date='2017/01/06'")
  .write
  .format("hudi")
  .options(getQuickstartWriteConfigs)
  .option(RECORDKEY_FIELD_OPT_KEY, "order_id")
  .option(PRECOMBINE_FIELD_OPT_KEY, "order_purchase_timestamp")
  .option(PARTITIONPATH_FIELD_OPT_KEY, "order_purchase_date")
  .option(TABLE_TYPE_OPT_KEY, "MERGE_ON_READ")
  .option(HIVE_SYNC_ENABLED_OPT_KEY, "true")
  .option(HIVE_PARTITION_FIELDS_OPT_KEY, "order_purchase_date")
  .option(HIVE_DATABASE_OPT_KEY, dbName)
  .option(HIVE_TABLE_OPT_KEY, tableName)
  .option(TABLE_NAME, tableName)
  .mode(Overwrite)
  .save(basePath)

// athena 에서 보시면 olist_orders_mor_ro, olist_orders_mor_rt 두개의 테이블이 생긴것을 보실 수 있습니다.
// <table_name>_rt - supports snapshot query and incremental query. base와 log 데이터를 합친 데이터를 보여줍니다.
//                   (HoodieParquetRealtimeInputFormat)
// <table_name>_ro - supports snapshot query and incremental query. base 데이터를 보여줍니다.
//                   HoodieParquetInputFormat

// Hudi 에서 쿼리 해 보겠습니다.
// _ro 테이블은 쿼리가 되나, _rt 테이블은 오류가 발생하는 것을 보실 수 있습니다.
// Athena는 현재는 Read Optimized (_ro)만 지원 합니다.

In [21]:
// 그렇다면 spark sql / hive / presto는 어떨까요?
println("**** olist_orders_mor_ro ****")
spark.sql("""
    select *
    from hudi.olist_orders_mor_ro
    where order_purchase_date = '2017-01-06'
""").show(false)

println("**** olist_orders_mor_rt ****")
spark.sql("""
    select *
    from hudi.olist_orders_mor_rt
    where order_purchase_date = '2017-01-06'
""").show(false)

// spark, hive 는 잘 되며, presto는 ro만 지원합니다.
// https://hudi.apache.org/docs/querying_data.html

**** olist_orders_mor_ro ****
+-------------------+--------------------+--------------------------------+----------------------+---------------------------------------------------------------------+--------------------------------+--------------------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key              |_hoodie_partition_path|_hoodie_file_name                                                    |order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_purchase_date|
+-------------------+--------------------+--------------------------------+----------------------+---------------------------------------------------------------------+--

In [12]:
// update

// 데이터 로드
// 한 2017년 1월 6일의 데이터를 로드 하도록 하겠습니다.
val olistOrdersDF = spark.sql("""
    select *
    from hudi.olist_orders_mor_ro
    where order_purchase_date = '2017-01-06'
""")

olistOrdersDF.show(false)

+-------------------+--------------------+--------------------------------+----------------------+---------------------------------------------------------------------+--------------------------------+--------------------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key              |_hoodie_partition_path|_hoodie_file_name                                                    |order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_purchase_date|
+-------------------+--------------------+--------------------------------+----------------------+---------------------------------------------------------------------+--------------------------------

In [13]:
// 그 중 한 건의 데이터에 대해 order_status를 ordered 로 바꾸도록 합니다. (그 전엔 delivered)
val updatedOlistOrdersDF = olistOrdersDF
  .where("order_id = '157ec3dc3f38cdbd2706bd216edfe8fb'")
  .withColumn("order_status", lit("ordered"))

// update 합니다.
// SaveMode가 Append 로 바뀐 것을 보실 수 있습니다.
updatedOlistOrdersDF
  .withColumn("order_purchase_date", regexp_replace(col("order_purchase_date"), "-", "/"))
  .write
  .format("hudi")
  .options(getQuickstartWriteConfigs)
  .option(RECORDKEY_FIELD_OPT_KEY, "order_id")
  .option(PRECOMBINE_FIELD_OPT_KEY, "order_purchase_timestamp")
  .option(PARTITIONPATH_FIELD_OPT_KEY, "order_purchase_date")
  .option(HIVE_SYNC_ENABLED_OPT_KEY, "true")
  .option(HIVE_PARTITION_FIELDS_OPT_KEY, "order_purchase_date")
  .option(HIVE_DATABASE_OPT_KEY, dbName)
  .option(HIVE_TABLE_OPT_KEY, tableName)
  .option(TABLE_NAME, tableName)
  .mode(Append)
  .save(basePath)

In [22]:
// 다시 한 번 2개의 테이블을 쿼리 해 보도록 하겠습니다.
println("**** olist_orders_mor_ro ****")
spark.sql("""
    select *
    from hudi.olist_orders_mor_ro
    where order_purchase_date = '2017-01-06'
""").show(false)

println("**** olist_orders_mor_rt ****")
spark.sql("""
    select *
    from hudi.olist_orders_mor_rt
    where order_purchase_date = '2017-01-06'
""").show(false)

// _ro 는 데이터가 변경되지 않고, _rt는 데이터가 변경 된 것을 보실 수 있습니다.
// 이쯤에서 S3의 변경 사항을 보셔야 하는데, 우선 commit 이 deltacommit 으로 변경 된 것을 보실 수 있습니다.
// 그리고 파티션에 해당하는 디렉토리를 보시면, log 파일이 생성 된 것을 보실 수 있습니다.
// 이 log 파일은 변경분에 대한 파일이며 avro로 인코딩 된 파일 입니다.

// 그러면 이쯤에서 compaction 에 대한 이야기를 꺼내야 할 것 같습니다.
// MOR 에서 지속적으로 log 파일들이 쌓이면서 성능도 안좋아질 것이기 때문에, 정리를 하여야 합니다.
// base + log가 결합 된 최신의 데이터를 parquet 파일로 변환하는 과정이고 이를 다시 base로 만드는 것입니다.

// inline으로(데이터를 ) 하는 방법과 명시적 command 등을 날려서 하는 방법이 있습니다.
// 시간 관계상 이라는 핑계를 대고 이 부분은 지금 하진 않겠습니다.
// inline은 아래의 페이지
// https://hudi.apache.org/docs/configurations.html#withInlineCompaction
// https://hudi.apache.org/docs/configurations.html#writeclient-configs
// command로 하는 부분은 아래의 페이지를 참고해 주세요.
// https://github.com/apache/hudi/blob/master/hudi-utilities/src/main/java/org/apache/hudi/utilities/HoodieCompactor.java

**** olist_orders_mor_ro ****
+-------------------+--------------------+--------------------------------+----------------------+---------------------------------------------------------------------+--------------------------------+--------------------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key              |_hoodie_partition_path|_hoodie_file_name                                                    |order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_purchase_date|
+-------------------+--------------------+--------------------------------+----------------------+---------------------------------------------------------------------+--